In [70]:
import subprocess
from pathlib import Path
import pandas as pd
from collections import Counter as count
import pydicom
import re
import io
import shutil

In [69]:
zips_path = Path('ullevaal_zips/')
dicom_path = Path('klinisk_baseline_ulleval_may2022/')
axial_ct_path = Path('ulleval_axial_cts/')
buffer = Path('buffer/')

In [3]:
ct_scans_99 = pd.read_table('99_ct_scans.txt')
ct_scans_90 = pd.read_table('90_ct_scans.txt')

In [4]:
ct_scans_99[['Patient name', 'Comments']]
ct_scans_90[['Patient name', 'Comments']]
a = set(ct_scans_99['Comments'])
b = set(ct_scans_90['Comments'])
missing_suids = a - b
missing_cts = pd.DataFrame({'StudyInstanceUID': list(missing_suids)})

In [5]:
missing_cts.to_csv('suid2.txt', sep=' ', index=False)

In [6]:
suid_input = 'suid2.txt'

In [7]:
zips = [z for z in zips_path.glob('*.zip')]

In [8]:
input_dicom_file_paths = [p for p in dicom_path.glob('**/*') if p.is_file()]

In [44]:
tag_search_term = re.compile('ax', re.IGNORECASE)

In [90]:
def filter_dicoms(path_to_dcm_file, pattern):
    dcm = pydicom.dcmread(path_to_dcm_file, stop_before_pixels=True, force=True)
    if isinstance(dcm, pydicom.dicomdir.DicomDir):
        pass
    elif isinstance(dcm, pydicom.dataset.FileDataset):
        if hasattr(dcm, 'Modality'): 
            if dcm.Modality == 'CT':
                if hasattr(dcm, 'ContrastBolusRoute'):
                    return
                if hasattr(dcm, 'ImageType'):
                    if dcm.ImageType[2] == 'LOCALIZER':
                        return 
                if hasattr(dcm, 'SeriesDescription'):
                    if pattern.search(dcm.SeriesDescription):
                        #series_ids.append(dcm.SeriesInstanceUID)
                        shutil.copy(path_to_dcm_file, buffer)

In [91]:
series_ids = []
for p in input_dicom_file_paths:
    filter_dicoms(p, tag_search_term)

In [92]:
series_uids = set(series_ids)
len(series_uids)

0

In [64]:
series_uids_list = list(series_uids)
series_uids_list.insert(0, 'SeriesInstanceUID')
string_input = '\n'.join(series_uids_list)
suid_txt = open('suid.txt', 'w')
suid_txt.write(string_input)
suid_txt.close()

In [65]:
#for zip in zips:
#    subprocess.call(['unzip', zip.as_posix(), '-d', dicom_path.as_posix()])

In [66]:
dicompull_location = '/Applications/dicomtools-0.8.13-mac/dicompull'

In [57]:
tags = ['ax til PACS', 'ax', 'Ax til PACS', 'Axial', 'AX', 'Caput', 'ax til pacs']

In [58]:
def run_dicompull(tag):
    subprocess.call([dicompull_location, '-k', 'Modality=CT', '-k', '0008,103e='+tag, dicom_path.as_posix(), '-o', axial_ct_path.as_posix()+'/{PatientID}/dicom/CT'])

In [67]:
def run_dicompull_suid(suids):
    subprocess.call([dicompull_location, '-u', suids, dicom_path.as_posix(), '-o', axial_ct_path.as_posix()+'/{PatientID}/dicom/CT'])

In [77]:
def dicompull_sort():
    subprocess.call([dicompull_location, buffer.as_posix(), '-o', axial_ct_path.as_posix()+'/{PatientID}/dicom/CT']) 

In [68]:
run_dicompull_suid('suid.txt')

Scanning 100%                      
Copying 100%                       


In [13]:
for tag in tags:
    run_dicompull(tag)

Scanning 100%                      
Scanning 100%                      
Scanning 100%                      
Scanning 100%                      
Scanning 100%                      
Scanning 100%                      
Scanning 100%                      


In [16]:
bla = [p for p in axial_ct_path.glob('*ct')]

In [17]:
len(bla)

97